In [ ]:
#!/usr/bin/env python3
"""
compare_cloud_masks.py

Comparaison entre :
- le masque nuage natif Landsat (fmask)
- et le masque externe de probabilité ciel clair (bayésian)

Objectifs :
- visualisation des masques
- application sur BT11
- statistiques d'accord sur pixels MER uniquement
"""

# ==========================================================
# 0) IMPORTS
# ==========================================================

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

# ==========================================================
# 1) FICHIERS D’ENTRÉE / SORTIE
# ==========================================================

# ---------- Landsat ----------
landsat_file = (
    "/home/kcorrea/Trishna/bayesian/"
    "LC09_L1C_GRIDDED_DATASET_202027_20221216_20230317_02_T1.nc"
)

# ---------- Masque externe ----------
# cfname = (
#     "LC09_L1C_GRIDDED_DATASET_203026_20221020_20230325_0_"
#     "clear_probability_x00.15_k50_deltaSST0.467_deltaTWCV=variable_w9_fallback5.tif"
# )
cloudmask_root = '/home/kcorrea/Trishna/bayesian/20221216/'
cfname = 'LC09_L1C_GRIDDED_DATASET_202027_20221216_20230317_0_clear_probability_x00.9_k40_deltaTWCV_variable_w15_fallback_v6f06.tif'
# cloudmask_file = "/home/kcorrea/Trishna/bayesian/20221216/" + cfname
cloudmask_file = cloudmask_root + cfname
# ---------- Sorties ----------
outdir = "/home/kcorrea/Trishna/figures/new/"
os.makedirs(outdir, exist_ok=True)

# ==========================================================
# 2) LECTURE LANDSAT
# ==========================================================

ds = xr.open_dataset(landsat_file)

cfmask = ds["provider_cloud_mask"]          # masque nuage natif Landsat
landmask = ds["land_mask"]                  # 1 = terre, 0 = mer
BT11 = ds["brightness_temperature_11"].squeeze()

lat = ds["latitude"].squeeze().values
lon = ds["longitude"].squeeze().values

BT = BT11.values
valid_BT = np.isfinite(BT)                  # hors-image

# ==========================================================
# 3) DÉCODAGE DU MASQUE NUAGE LANDSAT (FMASK)
# ==========================================================

def inspect_native_cloud_mask(cfarray, meanings):
    """
    Décodage du masque binaire fmask Landsat.

    Parameters
    ----------
    cfarray : xarray.DataArray
        provider_cloud_mask
    meanings : list of str
        types à considérer comme nuage

    Returns
    -------
    bool ndarray
        True = nuage
    """
    bit_dict = {
        "fill": 0,
        "dilated_cloud": 1,
        "cirrus": 2,
        "cloud": 3,
        "cloud_shadow": 4,
        "snow": 5,
        "clear": 6,
        "water": 7
    }

    mask = np.zeros(cfarray.shape, dtype=bool)

    for meaning in meanings:
        bit = bit_dict[meaning]
        mask |= (cfarray.astype(int) & (1 << bit)) != 0

    return mask


# Types considérés comme nuage
cloud_types = ["dilated_cloud", "cirrus", "cloud", "cloud_shadow", "snow" ]

mask_cloud_LS = inspect_native_cloud_mask(cfmask, cloud_types).squeeze()

# ==========================================================
# 4) CONSTRUCTION DU MASQUE LANDSAT FINAL
# ==========================================================

land = landmask.squeeze().values  # 1 = terre

# Codage :
#   0 = nuage
#   1 = clair
# NaN = terre / hors image
mask_LS = np.where(mask_cloud_LS, 0.0, 1.0)

mask_LS[land == 1] = np.nan
mask_LS[~valid_BT] = np.nan

# ==========================================================
# 5) LECTURE DU MASQUE EXTERNE (GeoTIFF)
# ==========================================================

cloud_ext = xr.open_dataarray(cloudmask_file).squeeze()

# Conversion en numpy + harmonisation
cloud_ext = cloud_ext.values.astype(float)

# On suppose :
#   1 = clair
#   0 = nuage
cloud_ext[land == 1] = np.nan
cloud_ext[~valid_BT] = np.nan

# Binarisation si besoin
cloud_ext_bin = np.where(cloud_ext >= 0.5, 1.0, 0.0)
cloud_ext_bin[np.isnan(cloud_ext)] = np.nan

# ==========================================================
# 6) VISUALISATIONS
# ==========================================================

# --- Masque Landsat ---
plt.figure(figsize=(8, 6))
plt.pcolormesh(lon, lat, mask_LS, cmap="bwr")
plt.colorbar(label="0 = cloud | 1 = clear")
plt.title("Landsat cloud mask (fmask)")
plt.tight_layout()
plt.savefig(outdir + cfname[0:-3] + "cloudmask_fmask.png", dpi=150)
plt.close()


In [ ]:
# --- BT11 avec masque Landsat ---
BT_cloud = BT.copy()
BT_cloud[mask_LS != 1] = np.nan

plt.figure(figsize=(8, 6))
plt.pcolormesh(lon, lat, BT_cloud, cmap="viridis", vmin=272, vmax=284)
plt.colorbar(label="BT11 (K)")
plt.title("BT11 with Landsat cloud mask")
plt.tight_layout()
plt.savefig(outdir + cfname[0:38] + "BT11_fmask.png", dpi=150)
plt.close()

In [ ]:
# --- Masque externe ---
plt.figure(figsize=(8, 6))
plt.pcolormesh(lon, lat, cloud_ext_bin, cmap="bwr")
plt.colorbar(label="0 = cloud | 1 = clear")
plt.title("External clear-sky mask")
plt.tight_layout()
plt.savefig(outdir + cfname[0:38] + "cloudmask_external.png", dpi=150)
plt.close()


In [ ]:
# --- BT11 avec masque bayesian ---
BT_cloud_ext = BT.copy()
BT_cloud_ext[cloud_ext_bin != 1] = np.nan

plt.figure(figsize=(8, 6))
plt.pcolormesh(lon, lat, BT_cloud_ext, cmap="viridis", vmin=272, vmax=284)
plt.colorbar(label="BT11 (K)")
plt.title("BT11 with bayesian cloud mask")
plt.tight_layout()
plt.savefig(outdir + cfname[0:38] + "BT11_bayesian.png", dpi=150)
plt.close()

In [ ]:
# ==========================================================
# 7) STATISTIQUES (MER UNIQUEMENT)
# ==========================================================

valid = np.isfinite(mask_LS) & np.isfinite(cloud_ext_bin)

LSv = mask_LS[valid]
EXTv = cloud_ext_bin[valid]

total = LSv.size

agreement = np.mean(LSv == EXTv) * 100
false_clear = np.mean((LSv == 1) & (EXTv == 0)) * 100
false_cloud = np.mean((LSv == 0) & (EXTv == 1)) * 100

print("\n=== OCEAN STATISTICS (Landsat = reference) ===")
print(f"Agreement        : {agreement:.2f} %")
print(f"False clear (FN) : {false_clear:.2f} %  (LS clear, EXT cloud)")
print(f"False cloud (FP) : {false_cloud:.2f} %  (LS cloud, EXT clear)")
print(f"Valid pixels     : {total}")


In [ ]:
# ==========================================================
# 8) CARTE DES DIFFÉRENCES
# ==========================================================

diff_mask = mask_LS - cloud_ext_bin

plt.figure(figsize=(8, 6))
plt.pcolormesh(lon, lat, diff_mask, cmap="bwr", vmin=-1, vmax=1)
plt.colorbar(label="fmask - external mask")
plt.title("Difference between masks")
plt.tight_layout()
plt.savefig(outdir + cfname[0:38] + "diff_fmask_vs_external.png", dpi=150)
plt.close()

print("\n Comparaison terminée")